First of all, set environment variables and initialize spark context:

In [ ]:
%env SPARK_DRIVER_MEMORY=8g
%env PYSPARK_PYTHON=/usr/bin/python3.5
%env PYSPARK_DRIVER_PYTHON=/usr/bin/python3.5

from zoo.common.nncontext import *
sc = init_nncontext(init_spark_conf().setMaster("local[4]"))

# CNN
First, let's take a practical look at a very simple convnet example. We will use our convnet to classify MNIST digits, a task that you've already been through in Chapter 2.

First load the MNIST dataset:

In [ ]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

#### CNN input shape
Once we get the dataset, we need to reshape the images. In keras the shape of the dataset is `(sample_size, height, width, channel)`, like the Keras code below:
    
    train_images = train_images.reshape((60000, 28, 28, 1))
In analytics-zoo, the default order is theano-style NCHW `(sample_size, channel, height, width)`, so you can process data like following:

Alternatively, you can also use tensorflow-style NHWC as Keras default just by setting `Convolution2D(dim_ordering="tf")`

In [ ]:
train_images = train_images.reshape((60000, 1, 28, 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 1, 28, 28))
test_images = test_images.astype('float32') / 255

from keras.utils.np_utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Then build the model, compile, train and evaluate:

In [ ]:
from zoo.pipeline.api.keras import layers
from zoo.pipeline.api.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, nb_col=3, nb_row=3, activation='relu', input_shape=(1,28,28)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, nb_col=3, nb_row=3, activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, nb_col=3, nb_row=3, activation='relu'))

model.summary()

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

test_result = model.evaluate(test_images, test_labels)
print('test_acc:', test_result[0].result)